# Inaugural Project

> **Note the following:** 
> 1. This is an example of how to structure your **inaugural project**.
> 1. Remember the general advice on structuring and commenting your code from [lecture 5](https://numeconcopenhagen.netlify.com/lectures/Workflow_and_debugging).
> 1. Remember this [guide](https://www.markdownguide.org/basic-syntax/) on markdown and (a bit of) latex.
> 1. Turn on automatic numbering by clicking on the small icon on top of the table of contents in the left sidebar.
> 1. The `inauguralproject.py` file includes a function which can be used multiple times in this notebook.

Imports and set magics:

In [2]:
import numpy as np

# autoreload modules when code is run. Otherwise, python will not see recent changes. 
%load_ext autoreload
%autoreload 2

# Import your own code
import inauguralproject

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Question 1

**Explain how you solve the model**

In [2]:
# code for solving the model (remember documentation and comments)
# b. call optimizer
from scipy import optimize
q_guess = [0]      
objective_function = lambda q: insured_expenditures(q)
res = optimize.minimize(objective_function, q_guess, method='Nelder-Mead')



[1 4 9]


We are now plotting the optimal insurance coverage:

In [ ]:
budgetline_x1 = np.linspace(0.001, I / p[0] , 100) # xs go up to the limit for good 1
budgetline_x2 = [(I - p[0] * x) / p[1] for x in budgetline_x1] # this is the tandard equation for the budget line

import matplotlib.pyplot as plt
x = np.linspace(0, 1)
plt.plot(x, , color='red')
plt.show())

fig = plt.figure()
ax = fig.add_subplot(1,1,1)
ax.plot(budgetline_x1, budgetline_x2, label = "Budgetline")

From this 

# Question 2

We are now going to find the set of acceptable contracts, where the expected value are the same for insured as not insured. 

In [3]:
# code

# Question 3

Explain your code and procedure

In [4]:
# code

ADD CONCISE CONLUSION.